In [129]:
#
# As variáveis de ambiente e globais abaixo precisam ser configuradas
# antes de rodar o notebook.
#

# Caminho para a chave do service account do GCP
# Veja: https://cloud.google.com/bigquery/docs/quickstarts/quickstart-client-libraries
%env GOOGLE_APPLICATION_CREDENTIALS=./google-cloud-key.json

# Credenciais para criancao de user
%env FULL_NAME=Marcela Barella
%env EMAIL=marcela.barella@hotmail.com

BQ_PROJECT="galvanic-smoke-385402"
BQ_DATASET="cadastro_produto"
BQ_TABLE="kh_data_engineer_teste_marcelabarella"

env: GOOGLE_APPLICATION_CREDENTIALS=./google-cloud-key.json
env: FULL_NAME=Marcela Barella
env: EMAIL=marcela.barella@hotmail.com


In [130]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install --upgrade 'google-cloud-bigquery[bqstorage,pandas]'

You should consider upgrading via the '/Users/grelinhodediamante/Projects/karhub/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/grelinhodediamante/Projects/karhub/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/grelinhodediamante/Projects/karhub/venv/bin/python3 -m pip install --upgrade pip' command.


In [131]:
%run -i 'karhub_extraction.py'

In [132]:
import json
import pandas as pd
import numpy as np

In [133]:
with open('./cars_data.json', 'r') as file:
    raw_data = json.load(file)

In [134]:
alias_df = pd.read_csv('karhub-alias.csv')

In [135]:
alias_df

,Fabricante,Código Fabricante,Alias
0,KarHub,FO0352,fo0352
1,KarHub,FO0353,fo0353
2,KarHub,FT0417,ft0417
3,KarHub,VW0127,vw0127
4,KarHub,PE0452,pe0452
...,...,...,...
1245,KarHub,I-VW0468,VW0468
1246,KarHub,I-CH0512,I-CH0512
1247,KarHub,I-CH0512,ich0512
1248,KarHub,I-MI0611,I-MI0611


In [136]:
alias_df.isnull().sum().sum()

0

In [137]:
autoparts_products_df = pd.read_excel('karhub_autoparts_1.xlsx')

In [138]:
autoparts_products_df

,Nome SKU,Fabricante,Código,Composição,Categoria,Nome Atributo,Valor Atributo
0,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Largura (cm),18.0
1,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Altura (cm),9.0
2,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Comprimento (cm),8.0
3,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Peso Bruto (Kg),0.600
4,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Peso Líquido (Kg),0.600
...,...,...,...,...,...,...,...
3313,Mola de Suspensão Dianteira Helicoidal,KarHub,I-VW0406,UNITARY,Mola Helicoidal,Largura (cm),40
3314,Mola de Suspensão Dianteira Helicoidal,KarHub,I-VW0406,UNITARY,Mola Helicoidal,Altura (cm),25
3315,Mola de Suspensão Dianteira Helicoidal,KarHub,I-VW0406,UNITARY,Mola Helicoidal,Comprimento (cm),25
3316,Mola de Suspensão Dianteira Helicoidal,KarHub,I-MI0605,UNITARY,Mola Helicoidal,Largura (cm),40


In [139]:
autoparts_products_df.isnull().sum().sum()

0

In [140]:
autoparts_products_df = autoparts_products_df.merge(alias_df[['Alias', 'Código Fabricante']], left_on='Código',
                                                    right_on='Alias', how='inner')

In [141]:
autoparts_products_df = autoparts_products_df.drop(['Código', 'Alias'], axis=1)

In [142]:
autoparts_products_df = autoparts_products_df.rename(columns={'Código Fabricante': 'Código'})

In [143]:
autoparts_products_df

,Nome SKU,Fabricante,Composição,Categoria,Nome Atributo,Valor Atributo,Código
0,Mola da Suspensão Traseira,KarHub,UNITARY,Mola Helicoidal,Largura (cm),18.0,FO0352
1,Mola da Suspensão Traseira,KarHub,UNITARY,Mola Helicoidal,Altura (cm),9.0,FO0352
2,Mola da Suspensão Traseira,KarHub,UNITARY,Mola Helicoidal,Comprimento (cm),8.0,FO0352
3,Mola da Suspensão Traseira,KarHub,UNITARY,Mola Helicoidal,Peso Bruto (Kg),0.600,FO0352
4,Mola da Suspensão Traseira,KarHub,UNITARY,Mola Helicoidal,Peso Líquido (Kg),0.600,FO0352
...,...,...,...,...,...,...,...
3295,Mola de Suspensão Dianteira Helicoidal,KarHub,UNITARY,Mola Helicoidal,Comprimento (cm),25,I-MI0611
3296,Mola de Suspensão Dianteira Helicoidal,KarHub,UNITARY,Mola Helicoidal,Peso Bruto (Kg),4.5,I-MI0611
3297,Mola de Suspensão Dianteira Helicoidal,KarHub,UNITARY,Mola Helicoidal,Peso Líquido (Kg),2.0,I-MI0611
3298,Mola de Suspensão Dianteira Helicoidal,KarHub,UNITARY,Mola Helicoidal,Posição,Dianteira,I-MI0611


In [163]:
pivoted_autoparts_products_df = autoparts_products_df.pivot_table(
    index='Código',
    columns='Nome Atributo',
    values='Valor Atributo',
    aggfunc='first'
)

In [164]:
pivoted_autoparts_products_df

Nome Atributo,Altura (cm),Comprimento (cm),Largura (cm),Motor,Número de Espirais,Peso Bruto (Kg),Peso Líquido (Kg),Posição
Código,,,,,,,,
FO0191,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira
FO0231,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira
FO0300,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira
FO0301,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira
FO0306,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Traseira
...,...,...,...,...,...,...,...,...
VW0446,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Traseira
VW0474,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira
VW0478,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira


In [165]:
pivoted_autoparts_products_df.reset_index(inplace=True)

In [166]:
pivoted_autoparts_products_df

Nome Atributo,Código,Altura (cm),Comprimento (cm),Largura (cm),Motor,Número de Espirais,Peso Bruto (Kg),Peso Líquido (Kg),Posição
0,FO0191,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira
1,FO0231,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira
2,FO0300,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira
3,FO0301,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira
4,FO0306,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Traseira
...,...,...,...,...,...,...,...,...,...
472,VW0446,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Traseira
473,VW0474,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira
474,VW0478,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira
475,VW0524,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira


In [167]:
pivoted_autoparts_products_df = pivoted_autoparts_products_df.merge(autoparts_products_df, on='Código', how='inner')

In [168]:
pivoted_autoparts_products_df

,Código,Altura (cm),Comprimento (cm),Largura (cm),Motor,Número de Espirais,Peso Bruto (Kg),Peso Líquido (Kg),Posição,Nome SKU,Fabricante,Composição,Categoria,Nome Atributo,Valor Atributo
0,FO0191,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,Largura (cm),18.0
1,FO0191,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,Altura (cm),9.0
2,FO0191,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,Comprimento (cm),8.0
3,FO0191,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,Peso Bruto (Kg),0.600
4,FO0191,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,Peso Líquido (Kg),0.600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3295,VW0525,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,Altura (cm),9.0
3296,VW0525,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,Comprimento (cm),8.0
3297,VW0525,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,Peso Bruto (Kg),0.600
3298,VW0525,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,Peso Líquido (Kg),0.600


In [169]:
pivoted_autoparts_products_df.groupby(['Código']).first()

,Altura (cm),Comprimento (cm),Largura (cm),Motor,Número de Espirais,Peso Bruto (Kg),Peso Líquido (Kg),Posição,Nome SKU,Fabricante,Composição,Categoria,Nome Atributo,Valor Atributo
Código,,,,,,,,,,,,,,
FO0191,9.0,8.0,18.0,None,None,0.600,0.600,Dianteira,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,Largura (cm),18.0
FO0231,9.0,8.0,18.0,None,None,0.600,0.600,Dianteira,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,Largura (cm),18.0
FO0300,9.0,8.0,18.0,None,None,0.600,0.600,Dianteira,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,Largura (cm),18.0
FO0301,9.0,8.0,18.0,None,None,0.600,0.600,Dianteira,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,Largura (cm),18.0
FO0306,9.0,8.0,18.0,None,None,0.600,0.600,Traseira,Mola da Suspensão Traseira,KarHub,UNITARY,Mola Helicoidal,Largura (cm),18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VW0446,9.0,8.0,18.0,None,None,0.600,0.600,Traseira,Mola da Suspensão Traseira,KarHub,UNITARY,Mola Helicoidal,Largura (cm),18.0
VW0474,9.0,8.0,18.0,None,None,0.600,0.600,Dianteira,Chicote eletrico,KarHub,UNITARY,Sonda Lambda,Largura (cm),18.0
VW0478,9.0,8.0,18.0,None,None,0.600,0.600,Dianteira,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal,Largura (cm),18.0


In [170]:
pivoted_autoparts_products_df.drop(['Nome Atributo', 'Valor Atributo'], axis=1, inplace=True)

In [171]:
pivoted_autoparts_products_df

,Código,Altura (cm),Comprimento (cm),Largura (cm),Motor,Número de Espirais,Peso Bruto (Kg),Peso Líquido (Kg),Posição,Nome SKU,Fabricante,Composição,Categoria
0,FO0191,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
1,FO0191,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
2,FO0191,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
3,FO0191,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
4,FO0191,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3295,VW0525,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
3296,VW0525,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
3297,VW0525,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal
3298,VW0525,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira,Mola da Suspensão Dianteira,KarHub,UNITARY,Mola Helicoidal


In [184]:
with open('./cars_data.json', 'r') as file:
    raw_data = json.load(file)

In [185]:
df_cars = pd.DataFrame.from_dict(raw_data)

In [186]:
df_cars.isnull().sum()

Fabricante             0
Código Fabricante      0
Marca                128
Modelo               128
Ano                  128
Complemento          128
dtype: int64

In [187]:
df_cars.fillna(np.nan).replace([np.nan], [None])

,Fabricante,Código Fabricante,Marca,Modelo,Ano,Complemento
0,KarHub,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V CLASS 2P | 1.0 MPI 8V - 4P | 1.0 ...
1,KarHub,FO0352,FORD,FIESTA,1997.0,1.0 MPI 8V - 2P | 1.3 MPI 8V CLX 2P | 1.3 MP...
2,KarHub,FO0352,FORD,FIESTA,1998.0,1.0 MPI 8V CLASS 2P | 1.0 MPI 8V CLASS 4P | ...
3,KarHub,FO0353,FORD,KA,1997.0,1.0i 8V - 2P | 1.3 MPI 8V CLX 2P | 1.0 MPI 8...
4,KarHub,FO0353,FORD,KA,1998.0,1.0i 8V - 2P | 1.3 MPI 8V CLX 2P | 1.0 MPI 8...
...,...,...,...,...,...,...
4210,KarHub,I-VW0468,AUDI,A3,2004.0,1.8 TURBO 20V 180CV - 4P | 1.6 8V - 2P | 1.8...
4211,KarHub,I-VW0468,AUDI,A3,2005.0,1.6 8V - 4P | 1.8 20V - 4P | 1.8 20V - 2P |...
4212,KarHub,I-VW0468,AUDI,A3,2006.0,1.8 20V - 4P | 1.6 8V - 4P | 1.8 TURBO 20V 1...
4213,KarHub,I-CH0512,None,None,None,None


In [188]:
df_cars = df_cars.assign(Complemento=df_cars['Complemento'].str.split('|'))

In [189]:
df_cars = df_cars.explode('Complemento')

In [190]:
df_cars

,Fabricante,Código Fabricante,Marca,Modelo,Ano,Complemento
0,KarHub,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V CLASS 2P
0,KarHub,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V - 4P
0,KarHub,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V - 2P
0,KarHub,FO0352,FORD,FIESTA,1996.0,1.3 MPI 8V CLX 4P
0,KarHub,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V CLASS 4P
...,...,...,...,...,...,...
4212,KarHub,I-VW0468,AUDI,A3,2006.0,1.8 TURBO 20V 150CV - 4P
4212,KarHub,I-VW0468,AUDI,A3,2006.0,1.8 20V - 2P
4212,KarHub,I-VW0468,AUDI,A3,2006.0,1.8 TURBO 20V 180CV - 4P
4213,KarHub,I-CH0512,None,None,NaN,None
